In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sys.path.insert(0, "../scripts/")
from spliting import SplitData
from sklearn import preprocessing


SyntaxError: invalid syntax (1045673435.py, line 7)

In [2]:
df =pd.read_csv('../data/cleaned_data.csv')
df.sample(10)

,auction_id,experiment,date,hour,device_make,platform_os,browser,user_response
1199,f1b2ff1e-f781-446e-a8c4-c53daea7e9fb,exposed,2020-07-06,5,VTR-L09,6,Facebook,0
1223,f9c4603c-5ccf-4470-8e63-d1edc6fe87b2,control,2020-07-07,1,Generic Smartphone,6,Chrome Mobile,0
147,4f071ba2-33aa-4fa2-b539-97d9bacc322c,control,2020-07-08,21,Samsung SM-A405FN,6,Facebook,1
614,1134407e-af87-434f-ac7c-299dff13dae2,control,2020-07-04,21,Samsung SM-A705FN,6,Facebook,0
1048,b501c220-43b8-42a5-a507-9db9bd659e96,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0
634,19548e79-bf25-4570-b8ca-54e8e28590f0,exposed,2020-07-03,1,Generic Smartphone,6,Chrome Mobile,0
297,9298f36a-b3b7-472e-923a-1cc1dbb919e7,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,1
1076,bfaa07e5-99ed-4975-b941-057d91629a61,exposed,2020-07-08,16,Samsung SM-A202F,6,Chrome Mobile WebView,0
1096,c8debff5-48be-4a6e-9a6b-eab3d676bdb8,control,2020-07-05,10,Samsung SM-G960F,6,Chrome Mobile WebView,0
497,e282a098-3437-4bf3-9819-f547f9ad9731,control,2020-07-08,21,Generic Smartphone,6,Chrome Mobile,1


In [10]:
browser_df=df.drop(['platform_os'],axis=1)
browser_df.head(10)


,auction_id,experiment,date,hour,device_make,browser,user_response
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,Chrome Mobile,1
1,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,Facebook,1
2,018af862-486e-4da1-a85b-71872120e57c,control,2020-07-03,15,Generic Smartphone,Chrome Mobile,1
3,023ec7b5-cb8f-49a5-995f-e0d7c2f702e5,exposed,2020-07-09,13,Samsung SM-G935F,Facebook,1
4,02efdb70-8596-4f3f-b0b2-b91e194f61f7,exposed,2020-07-05,6,Generic Smartphone,Chrome Mobile,1
5,0332a7f7-0336-4d48-96ad-75da00d916c0,control,2020-07-04,16,Generic Smartphone,Chrome Mobile,1
6,03948b84-b5b1-4f26-81fc-e953218aef1a,control,2020-07-09,20,Samsung SM-G930F,Facebook,1
7,03ee1780-1fa3-4bec-926c-d543a81a6653,exposed,2020-07-04,11,Samsung SM-G981B,Chrome Mobile WebView,1
8,046644f4-b30c-48f9-b069-f804a97b77d9,exposed,2020-07-05,2,Generic Smartphone,Chrome Mobile WebView,1
9,04b69bd8-4849-4489-adcf-08b96a265c40,control,2020-07-07,15,Generic Smartphone,Chrome Mobile,1


In [11]:
browser_df.to_csv('../data/browser_clean_data.csv')

In [12]:
platform_df=df.drop(['browser'],axis=1)
platform_df.sample(10)

,auction_id,experiment,date,hour,device_make,platform_os,user_response
23,0aaf0e53-0aef-48e1-86b8-77a751b6ad2b,control,2020-07-08,14,Generic Smartphone,6,1
979,9b7a2497-f2f3-4cfd-bf1b-bf533fa39f4f,exposed,2020-07-10,2,Generic Smartphone,6,0
416,c03a506f-92e4-41e1-8859-841d21f18349,exposed,2020-07-04,6,Samsung SM-G965F,6,1
789,577d8982-42dc-401d-8fc3-56e394a96782,exposed,2020-07-09,16,Generic Smartphone,6,0
889,7d6e7872-c491-4fa5-9436-87e13c3e32dc,exposed,2020-07-08,21,Samsung SM-G950F,6,0
754,4e013f88-ee69-49b8-955d-c661d491a450,exposed,2020-07-07,17,Samsung SM-G980F,6,0
369,ad5c63c0-72b0-404b-8a50-501f8246be93,exposed,2020-07-09,11,Generic Smartphone,6,1
987,9ea96c35-6b58-477a-bcca-392be377e13b,control,2020-07-10,3,VFD 820,6,0
350,a6d7e9ac-1bf2-4ae3-b7c5-92b26ab22c27,exposed,2020-07-04,13,F3311,6,1
494,e21bd15a-bd75-44d7-accc-20b3580f6055,control,2020-07-04,16,Generic Smartphone,6,1


In [13]:
platform_df.to_csv('../data/platform_clean_data.csv')

# Accessing DVC version

In [3]:
import dvc.api

<h3>Accessing Browser Data</h3>

In [16]:
with dvc.api.open(
    'data/browser_clean_data.csv',
    repo="https://github.com/abu-bakarr/10_Academy_Week2_Group",
    
) as f:
    browser_df = pd.read_csv(f)
    
browser_df.drop(['auction_id','Unnamed: 0'],axis=1,inplace=True)


In [17]:
with dvc.api.open(
    "data/platform_clean_data.csv",
    repo="https://github.com/abu-bakarr/10_Academy_Week2_Group",
) as fd:
    platform_df = pd.read_csv(fd)
    
platform_df.drop(["auction_id", "Unnamed: 0"], axis=1, inplace=True)


In [18]:
browser_df.sample(3)

,experiment,date,hour,device_make,browser,user_response
460,exposed,2020-07-09,18,Generic Smartphone,Chrome Mobile,1
648,exposed,2020-07-08,5,ELE-L29,Facebook,0
84,exposed,2020-07-09,22,Generic Smartphone,Chrome Mobile,1


In [54]:
def encode_labels(df):
    df['experiment'].replace(['control'],0,inplace=True)
    df["experiment"].replace(["exposed"], 1, inplace=True)
    df["date"]=preprocessing.LabelEncoder().fit_transform(df["date"])
    df["experiment"]=preprocessing.LabelEncoder().fit_transform(df["experiment"])
    df["hour"]=preprocessing.LabelEncoder().fit_transform(df["hour"])
    df["device_make"]=preprocessing.LabelEncoder().fit_transform(df["device_make"])
    
    return df

def encode_browser_labels(df):
    b_df=encode_labels(df)
    b_df["browser"]=preprocessing.LabelEncoder().fit_transform(df["browser"])
    return b_df
    
def encode_platform_labels(df):
    p_df=encode_labels(df)
    p_df["platform_os"]=preprocessing.LabelEncoder().fit_transform(df["platform_os"])
    return p_df
    


In [52]:
encoded_browser_df=encode_browser_labels(browser_df)
encoded_browser_df.sample(3)

,experiment,date,hour,device_make,browser,user_response
522,1,0,4,94,2,1
462,1,0,20,13,1,1
249,1,0,3,26,1,1


In [12]:
platform_df.sample(3)

,experiment,date,hour,device_make,platform_os,user_response
420,exposed,2020-07-09,20,Generic Smartphone,6,1
653,exposed,2020-07-05,8,Generic Smartphone,6,0
1215,exposed,2020-07-10,5,Samsung SM-G960F,6,0


In [55]:
encoded_platform_df = encode_platform_labels(platform_df)
encoded_platform_df.sample(3)


,experiment,date,hour,device_make,platform_os,user_response
69,1,1,8,27,1,1
525,1,6,21,13,1,1
601,1,7,16,13,1,0


<h5>Setting up training,validation and testing data</h5>

In [56]:
split_browser_data=SplitData(encoded_browser_df)

In [57]:
split_platform_data=SplitData(encoded_platform_df)

In [58]:
browser_data=split_browser_data.test_validate_train(0.7,0.2,'user_response')

(870, 5)
(870,)
(248, 5)
(248,)
(125, 5)
(125,)


In [59]:
browser_x_train=browser_data['x_train']
browser_y_train=browser_data['y_train']
browser_x_valid=browser_data['x_valid']
browser_y_valid=browser_data['y_valid']
browser_x_test=browser_data['x_test']
browser_y_test=browser_data['y_test']

In [60]:
browser_x_valid.head(3)

,experiment,date,hour,device_make,browser
30,1,0,16,13,1
29,0,0,15,66,2
28,1,0,3,13,1


In [61]:
platform_data = split_platform_data.test_validate_train(0.7, 0.2, "user_response")


(870, 5)
(870,)
(248, 5)
(248,)
(125, 5)
(125,)


In [62]:
platform_x_train =platform_data["x_train"]
platform_y_train =platform_data["y_train"]
platform_x_valid =platform_data["x_valid"]
platform_y_valid =platform_data["y_valid"]
platform_x_test = platform_data["x_test"]
platform_y_test = platform_data["y_test"]

In [63]:
platform_x_train.head(3)

,experiment,date,hour,device_make,platform_os
621,1,0,0,13,1
832,0,0,15,13,1
831,1,1,19,13,1


<h2>Training with each dataset</h2>